<a href="https://colab.research.google.com/github/qurbanovkenan77-tech/mgmt467-analytics-portfolio/blob/main/Labs/Lab_VertexAI_BigQuery_PromptsOnly_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-471119' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt-467-471119.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt-467-471119.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-467-471119/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-471119.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

In [ ]:

query_string = """
SELECT
    DISTINCT `Sub-Category` AS Sub_Category
FROM
    `mgmt-467-47888.lab1_foundation.superstore_clean`
WHERE
    Region = 'West'
ORDER BY
    Sub_Category ASC
LIMIT 100
"""
results_df = query_job.to_dataframe()
display(results_df)

,ship_mode,total_sales,total_profit,profit_margin,sales_weighted_discount,n_orders
0,Standard Class,124826.6615,-11910.0122,-0.095412,0.218796,190
1,Second Class,43693.7475,-3320.6799,-0.075999,0.201652,61
2,First Class,28800.7760,-1365.3665,-0.047407,0.187050,47
3,Same Day,9644.3470,-1129.4225,-0.117107,0.278146,21


In [ ]:
query_string = """
SELECT DISTINCT
  sub_category AS Sub_Category
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
WHERE region = 'West'
ORDER BY Sub_Category ASC
LIMIT 100
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


Reflection:
The first result didn’t match my expectations because it returned shipping metrics instead of a single-column list. The ambiguity came from column naming (Sub-Category vs. sub_category) and not clearly specifying “no aggregations.” Once I corrected the table/column names and restricted the output to one column, the query produced the expected result.

### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
  customer_id,
  SUM(profit) AS total_profit
FROM
  `mgmt-467-471119.lab1_foundation.superstore_clean`
GROUP BY
  customer_id
ORDER BY
  total_profit DESC
LIMIT 10
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,customer_id,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = """
SELECT
  Ship_Mode,
  COUNT(*) AS order_count
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
WHERE Category = 'Technology'
GROUP BY Ship_Mode
ORDER BY order_count DESC
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)


,Ship_Mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = """ WITH maxd AS (
  SELECT DATE_TRUNC(MAX(order_date), MONTH) AS max_month
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
),
bounds AS (
  -- pretend the "current month start" is the month AFTER the max month in data
  SELECT DATE_ADD(max_month, INTERVAL 1 MONTH) AS this_month_start
  FROM maxd
)
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
  SUM(sales) AS monthly_revenue
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`, bounds
WHERE DATE_TRUNC(order_date, MONTH) >= DATE_SUB(this_month_start, INTERVAL 12 MONTH)
  AND DATE_TRUNC(order_date, MONTH) <  this_month_start
GROUP BY year_month
ORDER BY year_month ASC """

query_job = client.query(query_string)
df_b1 = query_job.to_dataframe()
display(df_b1)

,year_month,monthly_revenue
0,2017-01,43971.3740
1,2017-02,20301.1334
2,2017-03,58872.3528
3,2017-04,36521.5361
4,2017-05,44261.1102
5,2017-06,52981.7257
6,2017-07,45264.4160
7,2017-08,63120.8880
8,2017-09,87866.6520
9,2017-10,77776.9232


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
query_string = """
SELECT
  sub_category,
  SUM(profit) AS total_profit
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
GROUP BY sub_category
HAVING SUM(profit) < 0
ORDER BY total_profit ASC
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)


,sub_category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
query_string = """
WITH dim_products AS (
  SELECT DISTINCT
    product_id,
    product_name,
    category,
    sub_category
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
),
fact_sales AS (
  SELECT
    product_id,
    SUM(sales) AS total_sales
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  GROUP BY product_id
)
SELECT
  p.product_id,
  p.product_name,
  p.category,
  p.sub_category,
  fs.total_sales
FROM fact_sales fs
JOIN dim_products p
  USING (product_id)
ORDER BY fs.total_sales DESC
LIMIT 25
"""

query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)


,product_id,product_name,category,sub_category,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,Technology,Copiers,61599.8240
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,Binders,27453.3840
2,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,Technology,Machines,22638.4800
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,Furniture,Chairs,21870.5760
4,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,Office Supplies,Binders,19823.4790
5,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,Office Supplies,Binders,19024.5000
6,TEC-CO-10001449,Hewlett Packard LaserJet 3310 Copier,Technology,Copiers,18839.6860
7,TEC-MA-10001127,HP Designjet T520 Inkjet Large Format Printer ...,Technology,Machines,18374.8950
8,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,Office Supplies,Binders,17965.0680
9,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,Office Supplies,Supplies,17030.3120


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

In [ ]:
query_string = """
WITH state_sales AS (
  SELECT
    region,
    state,
    SUM(sales) AS total_sales
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  GROUP BY region, state
),
ranked_state_sales AS (
  SELECT
    region,
    state,
    total_sales,
    RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
  FROM state_sales
)
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM ranked_state_sales
WHERE sales_rank <= 3
ORDER BY region, sales_rank
"""

query_job = client.query(query_string)
df_d1 = query_job.to_dataframe()
display(df_d1)


,region,state,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
query_string = """
WITH yr_sales AS (
  SELECT
    sub_category,
    EXTRACT(YEAR FROM order_date) AS yr,
    SUM(sales) AS sales
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  GROUP BY sub_category, yr
),
years_info AS (
  SELECT
    COUNTIF(yr = 2023) > 0 AS has2023,
    COUNTIF(yr = 2024) > 0 AS has2024,
    MAX(yr) AS max_year
  FROM yr_sales
),
pairs AS (
  SELECT
    CASE WHEN has2023 AND has2024 THEN 2023 ELSE max_year - 1 END AS y1,
    CASE WHEN has2023 AND has2024 THEN 2024 ELSE max_year     END AS y2
  FROM years_info
),
joined AS (
  SELECT
    s1.sub_category,
    s1.sales AS sales_y1,
    s2.sales AS sales_y2,
    (s2.sales - s1.sales) AS yoy_delta
  FROM yr_sales s1
  JOIN yr_sales s2
    ON s1.sub_category = s2.sub_category
  CROSS JOIN pairs p
  WHERE s1.yr = p.y1 AND s2.yr = p.y2
)
SELECT *
FROM joined
ORDER BY yoy_delta DESC
LIMIT 5
"""
query_job = client.query(query_string)
df_d_cte = query_job.to_dataframe()
display(df_d_cte)


,sub_category,sales_y1,sales_y2,yoy_delta
0,Phones,78962.030,105340.516,26378.486
1,Binders,49683.325,72788.045,23104.720
2,Accessories,41895.854,59946.232,18050.378
3,Appliances,26050.315,42926.932,16876.617
4,Copiers,49599.410,62899.388,13299.978


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [ ]:
query_string = """
WITH subcat_sales AS (
  SELECT
    region,
    sub_category,
    SUM(sales) AS total_sales
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  GROUP BY region, sub_category
),
ranked AS (
  SELECT
    region,
    sub_category,
    total_sales,
    ROW_NUMBER() OVER (
      PARTITION BY region
      ORDER BY total_sales DESC
    ) AS rn
  FROM subcat_sales
)
SELECT
  region,
  sub_category,
  total_sales
FROM ranked
WHERE rn = 1
ORDER BY region
"""
df_e1 = client.query(query_string).to_dataframe()
display(df_e1)


,region,sub_category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
query_string = """
WITH yearly AS (
  SELECT
    EXTRACT(YEAR FROM order_date) AS yr,
    SUM(sales) AS yearly_revenue
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  WHERE sub_category = 'Phones'
  GROUP BY yr
),
lagged AS (
  SELECT
    yr,
    yearly_revenue,
    LAG(yearly_revenue) OVER (ORDER BY yr) AS prev_revenue
  FROM yearly
)
SELECT
  yr AS year,
  yearly_revenue,
  prev_revenue,
  CASE
    WHEN prev_revenue IS NULL OR prev_revenue = 0 THEN NULL
    ELSE 100.0 * (yearly_revenue - prev_revenue) / prev_revenue
  END AS yoy_pct
FROM lagged
ORDER BY year ASC
"""
df_e2 = client.query(query_string).to_dataframe()
display(df_e2)


,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


In [ ]:
query_string = """
WITH yearly AS (
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  WHERE sub_category = 'Phones'
  GROUP BY year
),
with_lag AS (
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue) OVER (ORDER BY year) AS prev_revenue
  FROM yearly
)
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  ROUND(100.0 * SAFE_DIVIDE(yearly_revenue - prev_revenue, prev_revenue), 2) AS yoy_pct
FROM with_lag
ORDER BY year ASC;
"""

query_job = client.query(query_string)
df_e2 = query_job.to_dataframe()
display(df_e2)



,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.73
2,2016,78962.030,68313.702,15.59
3,2017,105340.516,78962.030,33.41


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
query_string = """
WITH mx AS (
  SELECT DATE_TRUNC(MAX(order_date), MONTH) AS max_month
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
),
bounds AS (
  SELECT
    DATE_SUB(DATE_ADD(max_month, INTERVAL 1 MONTH), INTERVAL 24 MONTH) AS start_month,
    DATE_ADD(max_month, INTERVAL 1 MONTH) AS end_month
  FROM mx
),
monthly AS (
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sales) AS monthly_revenue
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`, bounds
  WHERE segment = 'Corporate'
    AND DATE_TRUNC(order_date, MONTH) >= start_month
    AND DATE_TRUNC(order_date, MONTH) <  end_month
  GROUP BY month
)
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (
    ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS ma_3
FROM monthly
ORDER BY month ASC
"""
df_e3 = client.query(query_string).to_dataframe()
display(df_e3)


,month,monthly_revenue,ma_3
0,2016-01-01,9485.1240,9485.124000
1,2016-02-01,2866.8840,6176.004000
2,2016-03-01,14226.4730,8859.493667
3,2016-04-01,16177.3760,11090.244333
4,2016-05-01,18452.2580,16285.369000
5,2016-06-01,7788.4940,14139.376000
6,2016-07-01,13168.9190,13136.557000
7,2016-08-01,12943.4705,11300.294500
8,2016-09-01,17802.8123,14638.400600
9,2016-10-01,32130.5450,20958.942600


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
  File "/tmp/ipython-input-134885341.py", line 2
    FORMAT_DATE('%Y-%m', DATE_TRUNC(Order_Date, MONTH)) AS year_month,
    ^
IndentationError: unexpected indent

SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(Order_Date, MONTH)) AS year_month,
  SUM(Sales) AS monthly_revenue
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
GROUP BY year_month
ORDER BY year_month

Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

In [ ]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
  SUM(Sales) AS monthly_revenue
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
GROUP BY year_month
ORDER BY year_month
"""
df_f1 = client.query(query_string).to_dataframe()
display(df_f1)


,year_month,monthly_revenue
0,2014-01,14236.8950
1,2014-02,4519.8920
2,2014-03,55691.0090
3,2014-04,28295.3450
4,2014-05,23648.2870
5,2014-06,34595.1276
6,2014-07,33946.3930
7,2014-08,27909.4685
8,2014-09,81777.3508
9,2014-10,31453.3930


The column is named order_date (lowercase) in this schema, so referencing Order_Date fails. Switching to the exact column name fixes the error; the quick sanity check is SELECT ANY_VALUE(order_date) FROM \…superstore_clean LIMIT 1.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[query_string = """
WITH years_info AS (
  SELECT
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2023 THEN 1 ELSE 0 END) > 0 AS has2023,
    SUM(CASE WHEN EXTRACT(YEAR FROM order_date) = 2024 THEN 1 ELSE 0 END) > 0 AS has2024,
    MAX(EXTRACT(YEAR FROM order_date)) AS max_year
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
),
bounds AS (
  -- Prefer 2023→2024 if both exist; else fall back to (max_year-1 → max_year)
  SELECT 2023 AS y1, 2024 AS y2
  FROM years_info
  WHERE has2023 AND has2024
  UNION ALL
  SELECT max_year - 1 AS y1, max_year AS y2
  FROM years_info
  WHERE NOT (has2023 AND has2024)
),
yr_sales AS (
  SELECT
    sub_category,
    EXTRACT(YEAR FROM order_date) AS yr,
    SUM(sales) AS sales
  FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
  GROUP BY sub_category, yr
)
SELECT
  s1.sub_category,
  s1.sales AS sales_y1,
  s2.sales AS sales_y2,
  (s2.sales - s1.sales) AS yoy_delta
FROM yr_sales s1
JOIN yr_sales s2
  ON s1.sub_category = s2.sub_category
JOIN bounds b
  ON s1.yr = b.y1 AND s2.yr = b.y2
ORDER BY yoy_delta DESC
LIMIT 5
"""

query_job = client.query(query_string)
df_d2 = query_job.to_dataframe()
display(df_d2)]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

In [ ]:
'''
1) Partition filter early
Filter on the partitioned date column (order_date) to only the two target years (y1, y2) before any aggregation. If the table isn’t partitioned by order_date, do that (and optionally cluster by sub_category) to maximize pruning.

2) Column pruning
Read only the columns you actually need: order_date, sub_category, and sales. Avoid SELECT * and drop unused fields so BigQuery doesn’t scan those bytes.

3) Pre-aggregate with CTEs (and avoid self-joins)
Use a CTE to aggregate to (sub_category, year) for just the filtered years, then compute sales_y1 and sales_y2 via conditional aggregation (one grouped pass) rather than joining two aggregated tables. This shrinks the data early and eliminates join overhead.'''

'\n1) Partition filter early\nFilter on the partitioned date column (order_date) to only the two target years (y1, y2) before any aggregation. If the table isn’t partitioned by order_date, do that (and optionally cluster by sub_category) to maximize pruning.\n\n2) Column pruning\nRead only the columns you actually need: order_date, sub_category, and sales. Avoid SELECT * and drop unused fields so BigQuery doesn’t scan those bytes.\n\n3) Pre-aggregate with CTEs (and avoid self-joins)\nUse a CTE to aggregate to (sub_category, year) for just the filtered years, then compute sales_y1 and sales_y2 via conditional aggregation (one grouped pass) rather than joining two aggregated tables. This shrinks the data early and eliminates join overhead.'

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [ ]:
query_string = """
SELECT
  region,
  EXTRACT(YEAR FROM order_date) AS yr,
  SUM(sales)                                   AS total_sales,
  SUM(profit)                                  AS total_profit,
  SAFE_DIVIDE(SUM(profit), SUM(sales))         AS profit_margin,
  AVG(discount)                                AS avg_discount,
  COUNT(*)                                     AS n_orders
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
WHERE sub_category = 'Tables'
GROUP BY region, yr
ORDER BY region, yr
"""

query_job = client.query(query_string)
df_g1a = query_job.to_dataframe()
display(df_g1a)


,region,yr,total_sales,total_profit,profit_margin,avg_discount,n_orders
0,Central,2014,7785.4780,-1424.3310,-0.182947,0.326667,15
1,Central,2015,6857.2600,-265.0939,-0.038659,0.207143,14
2,Central,2016,13922.9260,292.6211,0.021017,0.205882,17
3,Central,2017,10589.3070,-2162.8466,-0.204248,0.292308,26
4,East,2014,10603.7040,-3537.8375,-0.333642,0.380000,20
5,East,2015,8884.8060,-2275.8641,-0.256152,0.373333,15
6,East,2016,7825.3280,-2306.7783,-0.294784,0.368182,22
7,East,2017,11825.9690,-2904.9002,-0.245637,0.373913,23
8,South,2014,9940.9445,1107.9902,0.111457,0.113636,11
9,South,2015,7370.6745,-2171.3765,-0.294597,0.218750,8


In [ ]:
query_string = """
SELECT
  ship_mode,
  SUM(sales)                                   AS total_sales,
  SUM(profit)                                  AS total_profit,
  SAFE_DIVIDE(SUM(profit), SUM(sales))         AS profit_margin,
  -- sales-weighted discount to avoid bias from small orders
  SAFE_DIVIDE(SUM(discount * sales), SUM(sales)) AS sales_weighted_discount,
  COUNT(*)                                     AS n_orders
FROM `mgmt-467-471119.lab1_foundation.superstore_clean`
WHERE sub_category = 'Tables'
GROUP BY ship_mode
ORDER BY total_profit ASC
"""

query_job = client.query(query_string)
df_g1b = query_job.to_dataframe()
display(df_g1b)


,ship_mode,total_sales,total_profit,profit_margin,sales_weighted_discount,n_orders
0,Standard Class,124826.6615,-11910.0122,-0.095412,0.218796,190
1,Second Class,43693.7475,-3320.6799,-0.075999,0.201652,61
2,First Class,28800.7760,-1365.3665,-0.047407,0.187050,47
3,Same Day,9644.3470,-1129.4225,-0.117107,0.278146,21


Check whether negative profit holds consistently across all regions/years or is concentrated in specific places/times—if some region-years show positive margins despite discounts, the blanket claim weakens. Then examine ship mode: if losses cluster in, say, Same Day/First Class while Standard is breakeven or positive, fulfillment costs (or rush-order discounting) may be the driver rather than “high discounts” alone. Compare avg vs sales-weighted discounts and profit margin; a strong negative relationship in only certain slices suggests targeted fixes (pricing or shipping policy) instead of a global “Tables is unprofitable” conclusion.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next


- Across the period analyzed, average arrival delay is about 9.5 minutes, on-time performance averages ~66%, and cancellations are around 2.5%, indicating generally reliable operations with room to improve punctuality. On-time rates vary by carrier, while load factor averages ~0.82, suggesting healthy demand and efficient seat utilization. The cost vs. revenue relationship is strongly positive, confirming that higher-cost routes typically generate higher revenue. I recommend targeted punctuality fixes on weaker carrier/route slices while preserving high-load-factor capacity, and tracking on-time %, average delay, and load factor weekly to confirm impact.
```

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```



---


**Note:** I executed the generated SQL in Colab using the BigQuery client rather than directly in BigQuery Console. The results are identical since both environments query the same dataset.





---
## Submission checklist
- [x] Kept prompts precise and reproducible  
- [x] Captured at least **one** CTE query and **one** window function query  
- [x] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [x] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---